## DPD Tracking Insights: Analyzing Shipment Status and Delivery Progress

In [3]:
import time
import random
import pandas as pd
import requests
import traceback
from tqdm import tqdm
tqdm.pandas()

from requests_html import HTMLSession, AsyncHTMLSession
import lxml
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [120]:
pd.options.display.max_columns=None
pd.set_option('display.max_rows',500)
pd.set_option('display.max_colwidth', None)

In [7]:
dpd_trackings = pd.read_csv('../dpddata/prime-label.csv', sep='\t')
dpd_trackings.sample(5)

,OrderDate.date,Order.id,Variation.number,OrderProperty.value,OrderProperty.value.1,Order.statusName,OrderItem.quantity,OrderParcelService.backendName,ShippingPackages.packageNumber,Unnamed: 9
1268,2022-03-16 14:06:56,598000,Acmos-08.072.187,106,028-4888536-3011524,[8.1] Storniert mit Gutschrift,4,DPD,9446317364355,NaN
1028,2022-10-22 22:08:42,679304,Beko-299602500,106,303-0386756-1686724,[8.1] Storniert mit Gutschrift,1,DPD,9446185623035,NaN
167,2022-01-14 18:27:21,572755,NaN,106,306-6871401-6698711,[8] Storniert ohne Gutschrift,1,DPD,9446171643046,NaN
18,2021-11-24 12:41:05,553077,NaN,106,302-3850713-0347547,[8.1] Storniert mit Gutschrift,1,DPD,9446168234600,NaN
666,2022-07-01 15:23:00,637069,NaN,106,305-5066678-2754711,[8.1] Storniert mit Gutschrift,1,DPD,9446180931131,NaN


In [ ]:
def extract_dpd_shipping_status(shipping_number):
    """
    Extracts the current shipping status of a DPD parcel given its tracking number.

    Args:
    shipping_number (str): The tracking number of the DPD parcel.

    Returns:
    str: The current status of the DPD parcel.
    """

    # url is taken from site inspection => networking
    url = "https://tracking.dpd.de/rest/plc/de_DE/0"
    # url = 'https://tracking.dpd.de/status/de_DE/parcel/0' # missing leading 0 in csv table
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()), options=chrome_options
    )
    try:
        
        driver.get(url + str(shipping_number))
        # wait for and click on popup message
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")
        # find the status in the response
        for state in json.loads(soup.find("body").text)["parcellifecycleResponse"][
            "parcelLifeCycleData"
        ]["statusInfo"]:
            if state["isCurrentStatus"] == True:
                driver.quit()
                return state["status"]

    except Exception as ex:
        traceback.print_exc()
        return "shipping number not trackable"


In [9]:
# create a session
session = HTMLSession()
#session = AsyncHTMLSession()

In [20]:
# define the function
def extract_status(shipping_number):
    """function to extract dpd shipping status ( javascript generated content) using requests-html"""

    # take the shipping number and create the url to request
    url = "https://tracking.dpd.de/rest/plc/de_DE/0" + str(shipping_number)

    # use the session to get the data
    r = session.get(url)

    # wait a random time to avoid being detected as a bot
    time.sleep(random.uniform(1, 4))

    # Render the page, up the number on scrolldown to page down multiple times on a page
    r.html.arender(sleep=1, keep_page=True, scrolldown=1)

    try:
        # loop through the json to find the current status
        for state in r.json()["parcellifecycleResponse"]["parcelLifeCycleData"][
            "statusInfo"
        ]:
            if state["isCurrentStatus"] == True:
                print(state["status"])
                return state["status"]
    except Exception as e:
        # if the shipping number is not trackable, return this information
        return "shipping number not trackable"


In [16]:
def format_table(dpd_trackings_df):
    """
    Formats a pandas DataFrame of DPD trackings by adding a 'date' column and a 'current_status' column.
    The 'date' column is created by converting the 'OrderDate.date' column to a datetime format.
    The 'current_status' column is initialized to 'Untrackable' for all rows.

    Args:
    dpd_trackings_df (pandas.DataFrame): A DataFrame of DPD trackings.

    Returns:
    pandas.DataFrame: The formatted DataFrame of DPD trackings.
    """
    # Create a datetime column
    dpd_trackings_df["date"] = pd.to_datetime(
        dpd_trackings_df["OrderDate.date"], infer_datetime_format=True
    )

    # Get all trackable indexes, hard code. Tracking data more than 6 month are untrackable  
    trackable_indexes_after_juli_2022 = dpd_trackings_df[
        (dpd_trackings_df.date.dt.year == 2022)
        & (dpd_trackings_df.date.dt.month > 7)
    ].index

    trackable_indexes_juli_2022 = dpd_trackings_df[
        (dpd_trackings_df.date.dt.year == 2022)
        & (dpd_trackings_df.date.dt.month == 7)
        & (dpd_trackings_df.date.dt.day > 11)
    ].index

    trackable_indexes = trackable_indexes_juli_2022.append(
        trackable_indexes_after_juli_2022
    )

    # Initialize all current_status values to 'Untrackable'
    dpd_trackings_df["current_status"] = "Untrackable"

    return trackable_indexes, dpd_trackings_df

trackable_indexes, dpd_trackings = format_table(dpd_trackings)
dpd_trackings

,OrderDate.date,Order.id,Variation.number,OrderProperty.value,OrderProperty.value.1,Order.statusName,OrderItem.quantity,OrderParcelService.backendName,ShippingPackages.packageNumber,Unnamed: 9,date,current_status
0,2021-03-06 00:49:08,439008,Magma-J-KSWK250,106,304-7940808-3152355,[8.1] Storniert mit Gutschrift,1,DPD,9446151723051,NaN,2021-03-06 00:49:08,Untrackable
1,2021-10-28 20:42:27,542906,Bahco-EX22XT9C,106,305-7846273-6869124,[8.1] Storniert mit Gutschrift,1,DPD,9446166526241,NaN,2021-10-28 20:42:27,Untrackable
2,2021-10-28 20:42:27,542906,NaN,106,305-7846273-6869124,[8.1] Storniert mit Gutschrift,1,DPD,9446166526241,NaN,2021-10-28 20:42:27,Untrackable
3,2021-10-29 11:04:19,543147,Stubai-S283907,106,304-9522034-5989903,[8.1] Storniert mit Gutschrift,1,DPD,9446166551273,NaN,2021-10-29 11:04:19,Untrackable
4,2021-10-29 11:04:19,543147,NaN,106,304-9522034-5989903,[8.1] Storniert mit Gutschrift,1,DPD,9446166551273,NaN,2021-10-29 11:04:19,Untrackable
...,...,...,...,...,...,...,...,...,...,...,...,...
1281,2022-10-08 13:04:42,673763,NaN,106,305-2682829-9561945,[8.1] Storniert mit Gutschrift,1,DPD,9446318984956,NaN,2022-10-08 13:04:42,Untrackable
1282,2022-10-08 19:04:00,674054,Heitmann-385-39,106,306-6811907-2267536,[8.1] Storniert mit Gutschrift,1,DPD,9446318985522,NaN,2022-10-08 19:04:00,Untrackable
1283,2022-10-08 19:04:00,674054,NaN,106,306-6811907-2267536,[8.1] Storniert mit Gutschrift,1,DPD,9446318985522,NaN,2022-10-08 19:04:00,Untrackable
1284,2022-10-12 13:08:37,675580,Picard-6006041019,106,302-9714888-1745958,[8.1] Storniert mit Gutschrift,1,DPD,9446319018050,NaN,2022-10-12 13:08:37,Untrackable


In [18]:
dpd_trackings.loc[trackable_indexes[:5],'current_status'] = dpd_trackings.loc[trackable_indexes[:5],'ShippingPackages.packageNumber'].progress_apply(extract_status)

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]/var/folders/b8/g_kq7hks5k514lxq387j61600000gn/T/ipykernel_39760/3777497657.py:12: RuntimeWarning: coroutine 'HTML.arender' was never awaited
  r.html.arender(sleep=1, keep_page=True, scrolldown=1)
100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.90s/it]


In [159]:
dpd_trackings.to_csv('../dpddata/dpd_tracking_with_current_status.csv')

In [160]:
dpd_trackings.current_status.value_counts()

Untrackable                            822
DELIVERED                              335
DATA_TRANSMITTED                        65
RETURN_TO_SENDER                        54
PICKUP_FROM_PARCELSHOP_BY_CONSIGNEE      8
OUT_FOR_DELIVERY                         2
Name: current_status, dtype: int64